In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier # 의사결정나무
from sklearn.linear_model import LogisticRegression # 로지스틱 회귀
from sklearn.ensemble import RandomForestClassifier

In [3]:
!pip install dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install dask[dataframe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 141 kB 4.7 MB/s 


In [5]:
import dask.dataframe as dd

In [6]:
# df = pd.read_csv("/content/drive/MyDrive/AMEX/data/train_data.csv", chunksize =500000)
ddf = dd.read_csv("/content/drive/MyDrive/AMEX/data/train_data.csv")

In [ ]:
df = ddf.compute()

In [ ]:
df = df.sort_values(['customer_ID','S_2'])
df = df.drop_duplicates(subset='customer_ID', keep='last')
train_labels =  pd.read_csv("/content/drive/MyDrive/AMEX/data/train_labels.csv") 

# Add labels 
df = df.merge(train_labels, on = 'customer_ID', how='left')

# Reponse column
y = "target"

# Features 목록
features = list(df.columns)
features.remove('customer_ID')
features.remove('S_2')
features.remove(y)

In [ ]:
df.isnull().sum()

In [ ]:
df=df.fillna(-1)

In [ ]:
df.isnull().sum()

In [ ]:
# df.to_parquet('/content/drive/MyDrive/AMEX/data/train.parquet')

### split

In [ ]:
# train = df.drop(columns=['customer_ID','S_2', 'target'])
# test = train['target']
# seed = 1004
# train_x, train_y, test_x, test_y = train_test_split(train, test, stratify=test, test_size = 0.4, random_state = seed)

In [ ]:
!pip install dask_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import dask_ml

In [ ]:
train, test = dask_ml.model_selection.train_test_split(df, test_size=0.3,random_state=1004)

/usr/local/lib/python3.7/dist-packages/dask_ml/model_selection/_split.py:469: FutureWarning: The default value for 'shuffle' must be specified when splitting DataFrames. In the future DataFrames will automatically be shuffled within blocks prior to splitting. Specify 'shuffle=True' to adopt the future behavior now, or 'shuffle=False' to retain the previous behavior.
  category=FutureWarning,


In [ ]:
seed = 1004
train, test = train_test_split(pandadf, test_size=0.4, random_state = seed)

In [ ]:
train_id = train['customer_ID']
train_x = train.drop(columns=['customer_ID','S_2', 'target'])
train_y = train['target']

In [ ]:
test_id = test['customer_ID']
test_x = test.drop(columns=['customer_ID','S_2', 'target'])
test_y = test['target']

In [ ]:
# pd.get_dummies(df, dummy_na=True)
train_x = pd.get_dummies(train_x)
test_x = pd.get_dummies(test_x)

# modeling

## modeling (LogisticRegression)

In [ ]:
lr = LogisticRegression(penalty='l2', C=10)
lr.fit(train_x, train_y)
train_score = lr.score(train_x, train_y)
test_score = lr.score(test_x, test_y)
print('train : ',train_score)
print('test : ',test_score)

NotImplementedError: ignored

In [ ]:
# from sklearn.model_selection import GridSearchCV

# params = {'penalty':['l2', 'l1'],
#          'C':[0.01, 0.1, 1, 5, 10]}

# grid_clf = GridSearchCV(lr, param_grid=params, scoring='accuracy', cv=3)
# grid_clf.fit(train_x, train_y)
# print('최적 하이퍼 파라미터 :{0}, 최적 평균 정확도 :{1:.3f}'.format(grid_clf.best_params_,
#                                                    grid_clf.best_score_))

## modeling (DecisionTree)

In [ ]:
tr = DecisionTreeClassifier()
tr.fit(train_x, train_y)
train_score = tr.score(train_x, train_y)
test_score = tr.score(test_x, test_y)
print('train : ',train_score)
print('test : ',test_score)

## modeling(RandomForest)

In [ ]:
rf = RandomForestClassifier(random_state=77)
rf.fit(train_x,train_y)
train_score = rf.score(train_x, train_y)
test_score = rf.score(test_x, test_y)
print('train : ',train_score)
print('test : ',test_score)

In [ ]:
rf = RandomForestClassifier(random_state=100, n_estimators=200)
rf.fit(train_x,train_y)
train_score = rf.score(train_x, train_y)
test_score = rf.score(test_x, test_y)
print('train : ',train_score)
print('test : ',test_score)

In [ ]:
rf = RandomForestClassifier(random_state=150, n_estimators=200)
rf.fit(train_x,train_y)
train_score = rf.score(train_x, train_y)
test_score = rf.score(test_x, test_y)
print('train : ',train_score)
print('test : ',test_score)

In [ ]:
esti = [150, 200, 250]
deeep = [10, 50, 100, 200]

for d in deeep:
  for n in esti:
    rf = RandomForestClassifier(random_state=77, n_estimators=n, max_depth=d)
    rf.fit(train_x,train_y)
    train_score = rf.score(train_x, train_y)
    test_score = rf.score(test_x, test_y)
    print('train : ',train_score, '/ n :',n,'/d : ',d)
    print('test : ',test_score)
  print()

# 예측값 구하기

In [ ]:
predict_lr = lr.predict(test_x)
predict_lr

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
predict_tr = tr.predict(test_x)
predict_tr

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
predict_rf = rf.predict(test_x)
predict_rf

array([0, 0, 0, ..., 1, 0, 0])

- 예측값

In [ ]:
test_x_lr = test_x.copy()
test_x_tr = test_x.copy()
test_x_rf = test_x.copy()

# 예측값 결과 비교 확인

In [ ]:
from sklearn.metrics import accuracy_score # 정확도 함수

In [ ]:
print('LogisticRegression : ',accuracy_score(test_y,predict_lr))
print()
print('DecisionTree : ',accuracy_score(test_y,predict_tr))
print()
print('RandomForest : ',accuracy_score(test_y,predict_rf))

LogisticRegression :  0.8916636506213054

DecisionTree :  0.8374351550247315

RandomForest :  0.8783930510314875


기본 모델에서는 Logistic - RandomForest - DecisionTree